In [1]:
import os
import argparse

import torch
import os
import argparse
import matplotlib.pyplot as plt

import torch
import torchvision.transforms as transforms

from os2d.modeling.model import build_os2d_from_config
from os2d.config import cfg
import  os2d.utils.visualization as visualizer
from os2d.structures.feature_map import FeatureMapSize
from os2d.utils import setup_logger, read_image, get_image_size_after_resize_preserving_aspect_ratio
from os2d.data import dataloader
from os2d.modeling.model import build_os2d_from_config

from os2d.data.dataloader import build_eval_dataloaders_from_cfg, build_train_dataloader_from_config
from os2d.engine.train import trainval_loop
from os2d.utils import set_random_seed, get_trainable_parameters, mkdir, save_config, setup_logger, get_data_path
from os2d.engine.optimization import create_optimizer
from os2d.config import cfg
from os2d.utils.visualization import *
import random
import os2d.utils.visualization as visualizer
from pathlib import Path
import cv2
import numpy as np
from os2d.utils import get_image_size_after_resize_preserving_aspect_ratio
from src.util.detection import generate_detection_boxes
from src.util.visualize import visualize_boxes_on_image
from src.util.filter import DataLoaderDB

### INIT

In [3]:
import torch
import os
import csv
import json
from datetime import datetime

def save_evaluation_to_csv(iter, results, filename='./src/util/evaluation_results.csv'):
    """
    將評估結果保存到 CSV 文件
    
    Args:
        results: 評估結果字典
        filename: CSV 文件名
    """
    
    # 確保目錄存在
    directory = os.path.dirname(filename)
    if directory and not os.path.exists(directory):
        os.makedirs(directory, exist_ok=True)
    
    # 提取主要指標
    overall_metrics = results['overall_metrics']
    bijection_stats = results['bijection_stats']
    eval_params = results['evaluation_params']
    
    # 計算額外統計
    class_metrics = results['class_metrics']
    num_classes = len(class_metrics)
    classes_with_perfect_score = sum(1 for metrics in class_metrics.values() if metrics['f1_score'] == 1.0)
    classes_with_zero_score = sum(1 for metrics in class_metrics.values() if metrics['f1_score'] == 0.0)
    
    # 準備 CSV 數據
    csv_data = {
        'iteration': str(iter),
        'timestamp': datetime.now().strftime('%Y-%m-%d %H:%M:%S'),
        'model_type': eval_params['model_type'],
        'evaluated_images': eval_params['evaluated_images'],
        'iou_threshold': eval_params['iou_threshold'],
        'precision': overall_metrics['precision'],
        'recall': overall_metrics['recall'],
        'f1_score': overall_metrics['f1_score'],
        'mean_iou': overall_metrics['mean_iou'],
        'map': overall_metrics['map'],
        'matching_rate': overall_metrics['matching_rate'],
        'total_detections': bijection_stats['total_detections'],
        'total_ground_truths': bijection_stats['total_ground_truths'],
        'total_matched_pairs': bijection_stats['total_matched_pairs'],
        'unmatched_detections': bijection_stats['unmatched_detections'],
        'unmatched_gts': bijection_stats['unmatched_gts'],
        'num_classes': num_classes,
        'classes_perfect_score': classes_with_perfect_score,
        'classes_zero_score': classes_with_zero_score
    }
    
    # 檢查文件是否存在以決定是否寫入標題
    file_exists = os.path.exists(filename)
    
    # 寫入 CSV
    with open(filename, 'a', newline='', encoding='utf-8') as csvfile:
        writer = csv.DictWriter(csvfile, fieldnames=csv_data.keys())
        
        # 如果文件不存在，寫入標題
        if not file_exists:
            writer.writeheader()
        
        # 寫入數據
        writer.writerow(csv_data)
    
    print(f"評估結果已保存到: {filename}")
    return filename

def show_gpu_memory_usage():
    """顯示當前 GPU RAM 使用情況"""
    if torch.cuda.is_available():
        device = torch.cuda.current_device()
        device_name = torch.cuda.get_device_name(device)

        # 獲取記憶體使用情況 (轉換為 GB)
        allocated = torch.cuda.memory_allocated(device) / 1024**3
        reserved = torch.cuda.memory_reserved(device) / 1024**3
        total = torch.cuda.get_device_properties(device).total_memory / 1024**3

        print(f"🖥️  GPU 設備: {device_name}")
        print(f"📊 記憶體使用情況:")
        print(f"   已分配: {allocated:.2f} GB")
        print(f"   已保留: {reserved:.2f} GB")
        print(f"   總容量: {total:.2f} GB")
        print(f"   使用率: {(allocated/total)*100:.1f}%")
        print(f"   保留率: {(reserved/total)*100:.1f}%")

        # 視覺化進度條
        usage_percent = int((allocated/total)*100)
        bar_length = 20
        filled_length = int(bar_length * usage_percent / 100)
        bar = '█' * filled_length + '░' * (bar_length - filled_length)
        print(f"   [{bar}] {usage_percent}%")

    else:
        print("❌ CUDA 不可用，無法檢測 GPU 記憶體")

import torch

def count_parameters(model):
    """
    計算模型的參數數量
    Returns:
      total_params: 包含所有參數
      trainable_params: 只包含 requires_grad=True 的參數
    """
    total_params = sum(p.numel() for p in model.parameters())
    trainable_params = sum(p.numel() for p in model.parameters() if p.requires_grad)
    return total_params, trainable_params

import io

def estimate_model_size(model):
    """
    將模型序列化到緩衝區，估算存檔大小（MB）
    """
    buffer = io.BytesIO()
    torch.save(model.state_dict(), buffer)
    size_mb = buffer.getbuffer().nbytes / (1024 ** 2)
    return size_mb

def show_network_status( orig_net, prune_net ):
    print( "原始網路參數統計:\n" )
    model = orig_net
    total, trainable = count_parameters(model)
    print(f"總參數量: {total:,}")
    print(f"可訓練參數量: {trainable:,}")

    size_mb = estimate_model_size(model)
    print(f"模型存儲大小: {size_mb:.2f} MB")

    print( "剪枝網路參數統計:\n" )
    model = prune_net
    total, trainable = count_parameters(model)
    print(f"總參數量: {total:,}")
    print(f"可訓練參數量: {trainable:,}")

    size_mb = estimate_model_size(model)
    print(f"模型存儲大小: {size_mb:.2f} MB")

In [4]:
if cfg.is_cuda:
    assert torch.cuda.is_available(), "Do not have available GPU, but cfg.is_cuda == 1"
    torch.backends.cudnn.benchmark = True

# random seed
set_random_seed(cfg.random_seed, cfg.is_cuda)

# Model
net, box_coder, criterion, img_normalization, optimizer_state = build_os2d_from_config(cfg)

# Optimizer
parameters = get_trainable_parameters(net)
optimizer = create_optimizer(parameters, cfg.train.optim, optimizer_state)

# load the dataset
data_path = get_data_path()
dataloader_train, datasets_train_for_eval = build_train_dataloader_from_config(cfg, box_coder, img_normalization,
                                                                                data_path=data_path)

dataloaders_eval = build_eval_dataloaders_from_cfg(cfg, box_coder, img_normalization,
                                                    datasets_for_eval=datasets_train_for_eval,
                                                    data_path=data_path)

db = DataLoaderDB( path = './src/db/data.csv' , dataloader = dataloader_train)

In [5]:
from src.lcp.ct_aoi_align import ContextAoiAlign
transform_image = transforms.Compose([
                      transforms.ToTensor(),
                      transforms.Normalize(img_normalization["mean"], img_normalization["std"])
                      ])

context_aoi_align = ContextAoiAlign( db, dataloader_train, transform_image , net , cfg )

In [6]:
from src.lcp.aux_net import AuxiliaryNetwork
aux_net = AuxiliaryNetwork( context_aoi_align, db )

In [7]:
from src.lcp.lcp import LCP
lcp = LCP(net, aux_net, dataloader_train)
lcp.init_for_indices()

[LCP] 初始化完成，共 43 層的 channel 索引


In [8]:
from src.util.prune_db import PruneDBControler
prune_db = PruneDBControler( path = './src/db/prune_channel_information-test-for_eval.csv' )
prune_db.initial()

檔案 ./src/db/prune_channel_information-test-for_eval.csv 已存在，將刪除並重新創建。
創建新檔案: ./src/db/prune_channel_information-test-for_eval.csv
檔案 ./src/db/prune_channel_information-test-for_eval.csv 初始化完成。


In [9]:
from src.lcp.pruner import Pruner
pruner = Pruner( lcp._prune_net )
pruner.set_prune_db( prune_db )

In [10]:
layers = lcp.get_layers_name()
print( layers )
layer_names = []
for name, ch in layers:
    if name.endswith('.conv1') or name.endswith('.conv2'):
        layer_names.append( name )

[('conv1', 64), ('layer1.0.conv1', 64), ('layer1.0.conv2', 64), ('layer1.0.conv3', 256), ('layer1.0.downsample.0', 256), ('layer1.1.conv1', 64), ('layer1.1.conv2', 64), ('layer1.1.conv3', 256), ('layer1.2.conv1', 64), ('layer1.2.conv2', 64), ('layer1.2.conv3', 256), ('layer2.0.conv1', 128), ('layer2.0.conv2', 128), ('layer2.0.conv3', 512), ('layer2.0.downsample.0', 512), ('layer2.1.conv1', 128), ('layer2.1.conv2', 128), ('layer2.1.conv3', 512), ('layer2.2.conv1', 128), ('layer2.2.conv2', 128), ('layer2.2.conv3', 512), ('layer2.3.conv1', 128), ('layer2.3.conv2', 128), ('layer2.3.conv3', 512), ('layer3.0.conv1', 256), ('layer3.0.conv2', 256), ('layer3.0.conv3', 1024), ('layer3.0.downsample.0', 1024), ('layer3.1.conv1', 256), ('layer3.1.conv2', 256), ('layer3.1.conv3', 1024), ('layer3.2.conv1', 256), ('layer3.2.conv2', 256), ('layer3.2.conv3', 1024), ('layer3.3.conv1', 256), ('layer3.3.conv2', 256), ('layer3.3.conv3', 1024), ('layer3.4.conv1', 256), ('layer3.4.conv2', 256), ('layer3.4.con

In [11]:
lcp.set_prune_db(prune_db)

In [12]:
from src.lcp.lcpfinetune import LCPFineTune
lcp_finetune = LCPFineTune(
    prune_net = lcp._prune_net,
    dataloader_train = dataloader_train,
    img_normalization = img_normalization,
    box_coder = box_coder,
    cfg       = cfg,
    optimizer=optimizer,
    parameters=parameters
)
lcp_finetune._setup_logging()

#### Try one layer for channel selection computing for Test SetUp

In [14]:

for name, ch in layers:
    if name == 'layer2.0.conv2':
        pass
    else:
        continue
    print(f"{name}: {ch} channels")
    keep, discard = lcp.get_channel_selection_by_no_grad(
        layer_name   = f"net_feature_maps.{name}",
        discard_rate = 0.5,
        lambda_rate  = 1.0,
        use_image_num= 3,
        random_seed  = 42
    )
    print(f"layer {name} , 預計保留通道數量: {len(keep)}/{ch}, 預計捨棄通道數量: {len(discard)}/{ch}")

layer2.0.conv2: 128 channels
[LCP] 開始基於數學推導的無梯度通道重要性計算 - net_feature_maps.layer2.0.conv2
[398, 234, 35]
[LOG] 原始網路特徵提取完成
[LOG] 剪枝網路特徵提取完成
398 {'channels': [{'l1_norm': 0.4974992275238037, 'variance': 0.2507721781730652, 'mean_deviation': 0.3170994818210602, 'energy': 0.4189516305923462, 'sparsity': 0.0, 'importance': 0.42683321237564087}, {'l1_norm': 0.5611555576324463, 'variance': 0.2151084542274475, 'mean_deviation': 0.40230420231819153, 'energy': 0.46043458580970764, 'sparsity': 0.0, 'importance': 0.4581121504306793}, {'l1_norm': 1.7944762706756592, 'variance': 0.27135947346687317, 'mean_deviation': 1.875490427017212, 'energy': 3.4486026763916016, 'sparsity': 0.0, 'importance': 1.4066091775894165}, {'l1_norm': 1.482433795928955, 'variance': 0.2149149477481842, 'mean_deviation': 1.5592610836029053, 'energy': 2.3648128509521484, 'sparsity': 0.0, 'importance': 1.1314866542816162}, {'l1_norm': 3.641723871231079, 'variance': 0.1759093850851059, 'mean_deviation': 3.548717737197876, 'energ

### Main For Prune + Finetune

##### Set Up for config

In [17]:
cfg.defrost()

cfg.train.optim.max_iter = 1000
cfg.train.do_training = True
cfg.output.print_iter = 500
cfg.eval.iter = 200
cfg.train.batch_size = 8

cfg.freeze()


In [18]:
from src.util.loss import LCPFinetuneCriterion
lcp_criterion = LCPFinetuneCriterion(
    original_criterion=criterion,  # 原始損失函數實例
    aux_net=lcp._aux_net,  # 包含 aux_loss 方法的實例
    auxiliary_weight=1.0  # 可以調整輔助損失權重
)

#### Load Pruned Network

In [20]:
# from src.lcp.recontruction import LCPReconstruction
# lcp_reconstruction = LCPReconstruction(
#     prune_db = prune_db,
#     pruner = pruner,
#     prune_net = lcp._prune_net
# )

In [21]:
# lcp_reconstruction.load_checkpoint_with_pruned_net(
#     './src/util/checkpoints-test/checkpoint_lcp_finetune_26_layer3.5.conv2.pth'
# )
# lcp._prune_net = lcp_reconstruction._prune_net
# pruned_layers = []
# for layer in lcp_reconstruction._pruned_layers:
#     if layer not in pruned_layers:
#         pruned_layers.append(layer)
# for layer in pruned_layers:
#     lcp.prune_layer(
#         layer_name   = layer,
#         discard_rate = None,
#     )

##### Main code for LCP Algorithm 1

In [23]:
eval_results = lcp.evaluate(
        dataloader_train,
        img_normalization,
        box_coder,
        cfg,
        count=500
    )

filename = save_evaluation_to_csv( f"before_prune_iter_in_0" , eval_results)


開始評估 500 張圖像...
Image 172 size FeatureMapSize(w=2448, h=3264) has 16 boxes


c:\Users\marti\anaconda3\envs\ntut-project\lib\site-packages\torch\functional.py:504: UserWarning: torch.meshgrid: in an upcoming release, it will be required to pass the indexing argument. (Triggered internally at C:\cb\pytorch_1000000000000\work\aten\src\ATen\native\TensorShape.cpp:3484.)
  return _VF.meshgrid(tensors, **kwargs)  # type: ignore[attr-defined]


[((0.7863983318748797, 0.3527693186841441), (1.0, 0.5557800011051169)), ((0.04595550231698663, 0.3559066384824264), (0.3410387751320126, 0.5365888141869948))]
converted_boxes: [((0.7863983318748797, 0.3527693186841441), (1.0, 0.5557800011051169)), ((0.04595550231698663, 0.3559066384824264), (0.3410387751320126, 0.5365888141869948))], tpoints: [((0.0766999986436632, 0.3425000134636374), (0.33000001097037124, 0.5224999820484835)), ((0.0017000000461254243, 0.3638000114291322), (0.08669999689837686, 0.5224999820484835))]
Image 493 size FeatureMapSize(w=3264, h=2448) has 10 boxes
[((0.7836131741213193, 0.6835403736046959), (0.9922459978980225, 0.8012935882168977)), ((0.7688153203034787, 0.5307037377008756), (0.9747576636475174, 0.6491084638423787))]
converted_boxes: [((0.7836131741213193, 0.6835403736046959), (0.9922459978980225, 0.8012935882168977)), ((0.7688153203034787, 0.5307037377008756), (0.9747576636475174, 0.6491084638423787))], tpoints: [((0.7825000239353553, 0.683299993377885), (0

In [ ]:
for idx, layer_name in enumerate(layer_names):
    if layer_name == 'layer3.0.conv1' or layer_name == 'layer3.1.conv1' or layer_name == 'layer3.2.conv1' or layer_name == 'layer3.3.conv1' or layer_name == 'layer3.4.conv1' or layer_name == 'layer3.5.conv1':
        continue
    if idx > 13:
        lcp.prune_layer(
            layer_name   = layer_name,
            discard_rate = 0.35,
        )
    else:
        lcp.prune_layer(
            layer_name   = layer_name,
            discard_rate = 0.85,
        )
    lcp_finetune.start_finetune(
        criterion= lcp_criterion,  # 使用自定義損失函數
    )
    show_gpu_memory_usage()
    print(f"剪枝後網路狀態 (第 {idx+1}, {layer_name} 層):")
    show_network_status(lcp._net, lcp_finetune._prune_net)
    lcp._prune_net = lcp_finetune._prune_net  # 更新剪枝網路
    lcp.save_checkpoint_with_pruned_net(
        log_path = './src/util/checkpoints-test4',
        optimizer = optimizer,
        model_name = f"lcp_finetune_{idx+1}_{layer_name}",
        i_iter = cfg.train.optim.max_iter
    )
    lcp.debug_for_test_vision(
        dataloader_train = dataloader_train,
        img_normalization = img_normalization,
        box_coder = box_coder,
        cfg = cfg,
        count = 2
    )
    from PIL import Image

    # 讀取圖片
    img = Image.open("./visualized_images/image_None_.png")
    
    # 存成新檔名
    img.save(f"./visualized_images/{layer_name}.png")

    eval_results = lcp.evaluate(
        dataloader_train,
        img_normalization,
        box_coder,
        cfg,
        count=100
    )

    filename = save_evaluation_to_csv( f"{layer_name}_iter_{cfg.train.optim.max_iter*(idx+1)}" , eval_results )
    


[LCP] 開始基於數學推導的無梯度通道重要性計算 - net_feature_maps.layer1.0.conv1
[398, 234, 35]
[LOG] 原始網路特徵提取完成
[LOG] 剪枝網路特徵提取完成
398 {'channels': [{'l1_norm': 3.0341808795928955, 'variance': 0.247497096657753, 'mean_deviation': 1.8420730829238892, 'energy': 9.45374870300293, 'sparsity': 0.0, 'importance': 1.2299062013626099}, {'l1_norm': 3.039808511734009, 'variance': 0.6785176396369934, 'mean_deviation': 1.8477007150650024, 'energy': 9.91894817352295, 'sparsity': 0.0, 'importance': 1.2746195793151855}, {'l1_norm': 1.5199832916259766, 'variance': 0.19905774295330048, 'mean_deviation': 0.3277449607849121, 'energy': 2.5090084075927734, 'sparsity': 0.0, 'importance': 0.5399119257926941}, {'l1_norm': 0.7009216547012329, 'variance': 0.07953083515167236, 'mean_deviation': 1.8930294513702393, 'energy': 0.5708214044570923, 'sparsity': 0.0, 'importance': 0.4881601333618164}, {'l1_norm': 3.4120376110076904, 'variance': 6.929659843444824, 'mean_deviation': 2.031601905822754, 'energy': 17.321914672851562, 'sparsity':

C:\Users\marti\os2d-in-Channel-Prune\os2d\engine\objective.py:275: UserWarning: Number of positives in a batch cannot be zero, can't normalize this way, setting num_pos to 1
  warnings.warn("Number of positives in a batch cannot be zero, can't normalize this way, setting num_pos to 1")


[LCP Debug] Computing auxiliary loss...
[LCP Debug] batch_idx is None: False
[LCP Debug] aux_net is None: False
[Memory] 函數開始: 已分配 4.74 GB, 已保留 5.77 GB
[Memory] 獲取 image_ids 後: 已分配 4.74 GB, 已保留 5.77 GB
Image 457 size FeatureMapSize(w=3264, h=2448) has 13 boxes
Image 263 size FeatureMapSize(w=3264, h=2448) has 5 boxes
Image 127 size FeatureMapSize(w=3264, h=2448) has 13 boxes
Image 59 size FeatureMapSize(w=3264, h=2448) has 14 boxes
[Memory] 批次資料解包後: 已分配 4.74 GB, 已保留 4.95 GB
[LCP Debug] class_ids type: <class 'list'>
[LCP Debug] class_ids length: 15
[LCP Debug] class_ids[0] type: <class 'numpy.int64'>
[Memory] 處理 image_id 0: 已分配 4.74 GB, 已保留 4.95 GB
[LCP Debug] Sample 1: aux_loss = 4.470131
[Memory] 批次資料清理後: 已分配 4.74 GB, 已保留 4.95 GB
[LCP Debug] Total samples processed: 1/4
[LCP Debug] Average auxiliary loss: 4.470131
[Memory] 函數結束: 已分配 4.74 GB, 已保留 4.95 GB
[Memory] 最終清理後: 已分配 4.74 GB, 已保留 4.95 GB
[LCP Loss] Original loss: 0.524249 (valid: True)
[LCP Loss] Auxiliary loss: 4.470131 (valid

In [ ]:
lcp.debug_for_test_vision(
    dataloader_train = dataloader_train,
    img_normalization = img_normalization,
    box_coder = box_coder,
    cfg = cfg,
    count = 1
)

##### The Global Finetune

In [ ]:
show_network_status(lcp._net, lcp._prune_net)

In [ ]:
# global_finetune_iter = 20000
# cfg.defrost()
# cfg.train.optim.max_iter = 100
# cfg.freeze()

In [ ]:
# for iter in range( int(global_finetune_iter / cfg.train.optim.max_iter) ):
#     print(f"Global Finetune Iteration: {iter+1}/{int(global_finetune_iter / cfg.train.optim.max_iter)}")
#     lcp_finetune.start_finetune(
#         criterion= lcp_criterion,  # 使用自定義損失函數
#     )
#     lcp.save_checkpoint_with_pruned_net(
#         log_path = './src/util/checkpoints-test3',
#         optimizer = optimizer,
#         model_name = f"lcp_finetune_{iter+1}_global",
#         i_iter = cfg.train.optim.max_iter
#     )
#     lcp._prune_net = lcp_finetune._prune_net  # 更新剪枝網路
#     lcp.debug_for_test_vision(
#         dataloader_train = dataloader_train,
#         img_normalization = img_normalization,
#         box_coder = box_coder,
#         cfg = cfg,
#         count = 2
#     )